# MoCapRasp arena calibration and testing 
This notebook is a **visual debug tool** to **calibrate and test extrinsics parameters** of the eROBOTICA visual tracking arena. 

**IMPORTANT NOTES**

- The CSV files `camCalib.csv`, `camGround.csv` and `camTest.csv` must be collected via the `calib.py`, `ground.py` and `capture.py` codes, respectively (with the `--save` flag). 

- Please just change the variables in the noted cells only.

- All the non-essential and debug only image plots have been commented, but not removed. Feel free to explore them.

- The calibration of intrinsics has already been done with [this repository](https://github.com/debOliveira/myCameraCalibrator) and stored in the file `constants.py`. Please alter the matrices accordingly. 

- All the mathematical basis of this code can be found on the thesis text.

## Configuration 
Just importing libraries and self made functions used in the previous offline arena doodle.

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import time,math
import numpy as np
from constants import cameraMat
from cv2 import circle,triangulatePoints,cvtColor,COLOR_GRAY2RGB,computeCorrespondEpilines,putText,FONT_HERSHEY_SIMPLEX,imwrite,Rodrigues
from myLib import isCollinear,isEqual,myProjectionPoints,orderCenterCoord,occlusion,getOrderPerEpiline,findPlane,drawlines,findRandT,getTheClosest,myInterpolate,getOtherValidIdx,createNeedsOrder,activateNeedsOrder,popNeedsOrder,estimateFundMatrix_8norm,decomposeEssentialMat,findOtherCamera
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import CubicSpline
import os
from itertools import combinations,permutations

## Calibration of extrinsincs

The following cell sets up the name of the CSV where the data is stored and the size of the arrays to be read. Alter to the values used with the `calib.py` code.

In [ ]:
##########################################
### YOU MAY CHANGE THE VARIABLES BELOW ###
##########################################

# name (folder/camCalib.csv) where the CSV from calib.py is saved
dfCSV = np.genfromtxt('/home/debora/Desktop/MoCapRasp/server/data/camCalib.csv', delimiter=',')
verbose = False # verbose flag for reading the data and interpolation 
nCameras = 4    # number of cameras
recTime = 180    # recording time, in seconds
step = 1/100    # step of the interpolated DF

### Reading coordinates and ordering markers
Now, we read each line of the CSV that has the following strings:
- *`Ax|Ay|Bx|By|Cx|Cy`*: the x and y undistorted coordinates of each collinear marker A, B or C. 
- *`time`*: the PTS in seconds from when the image was captured.
- *`image number`*: the number of the image in the processing loop at the Raspberry station.
- *`index`*: the number the Raspberry station (varies from 0 to `nCameras`).

Then we check if there was no processing error in the data (e.g. if there is occlusion between markers or PTS divergion). Finally, we check how many images have been missed comparing to the image number of the last valid image from the stream. Note that we may receive delayed packages between Raspberries, but they come always in order from each stream station.

*The markers are ordered using the certainty algorithm proposed in this dissertation.* The timestamp period in which we locate the valid images of each camera's stream is recorded in a separate array and later used for interpolation.

In [ ]:
# initiating empty arrays
counter,i,lastTime = np.zeros(nCameras,dtype=np.int32),0,np.zeros(nCameras,dtype=np.int32)
missed,invalid,swap = np.zeros(nCameras,dtype=np.int32),np.zeros(nCameras,dtype=np.int32),np.zeros(nCameras,dtype=np.int32)
lastImgNumber = np.zeros(nCameras,dtype=np.int32)
certainty,intervals,timeIntervals = np.zeros(nCameras,dtype=np.bool8),[],[]
dfOrig,tol = [],0.25
for k in range(nCameras):
    dfOrig.append([])
    intervals.append([])
    timeIntervals.append([])

# reading the CSV per line
while i!=dfCSV.shape[0]:
    line = dfCSV[i]
    # get data from the line
    idx = int(line[8])
    undCoord,timeNow,imgNumber = line[0:6].reshape(-1,2),line[6],line[7]
    i+=1
    # if the timestamp has diverged, discard the image (processing error)
    if counter[idx]:
        if abs(timeNow-lastTime[idx])>1e9: 
            if verbose: print('time missmatch')
            missed[idx]+=1
            invalid[idx]+=1
            continue
    # check if no image was missed
    if imgNumber>lastImgNumber[idx]+1: invalid[idx] = imgNumber-lastImgNumber[idx]
    # check collinearity, occlusion and order markers per proximity
    if isCollinear(*undCoord) and not isEqual(undCoord,5) and not np.any(undCoord<0):     
        if invalid[idx]>=10 or not counter[idx]: 
            if certainty[idx]:
                beg,end = intervals[idx][-1],counter[idx]-1
                timeIntervals[idx].append([dfOrig[idx][beg,6],dfOrig[idx][end,6]])
                if verbose: print('camera #'+str(idx)+' valid from '+str(round(dfOrig[idx][beg,6]/1e6,2))+'s to '+str(round(dfOrig[idx][end][6]/1e6,2))+'s')
            prev,certainty[idx] = [],False
            intervals[idx].append(counter[idx])
        else:
            if not (counter[idx]-1): prev = np.array(dfOrig[idx][0:6]).reshape(-1,2)
            else: prev = np.array(dfOrig[idx][-1,0:6]).reshape(-1,2)
        undCoord, _ = orderCenterCoord(undCoord,prev)
        undCoord = np.array(undCoord)
    else: 
        if verbose: print('not collinear or equal centroids')
        missed[idx]+=1
        invalid[idx]+=1
        continue
    # update datasets variables
    lastTime[idx],lastImgNumber[idx],invalid[idx] = timeNow,imgNumber,0
    if not counter[idx]: dfOrig[idx] = np.hstack((undCoord.reshape(6),timeNow))
    else: dfOrig[idx] = np.vstack((dfOrig[idx],np.hstack((undCoord.reshape(6),timeNow))))
    counter[idx]+=1
    # check if ABC is in order smaller to largest, change if not (but only if certain of the distribution = if at least two images indicate that yes)
    if not certainty[idx]:
        for [A,B,C] in undCoord.reshape([-1, 3, 2]):
            if np.linalg.norm(A-B)/np.linalg.norm(C-B)>(2-tol) and np.linalg.norm(A-B)>20:
                swap[idx] += 1
                if swap[idx]>2:    
                    swap[idx],certainty[idx] = 0,True
                    dfOrig[idx][intervals[idx][-1]:counter[idx],0:2],dfOrig[idx][intervals[idx][-1]:counter[idx],4:6] = np.copy(dfOrig[idx][intervals[idx][-1]:counter[idx],4:6]),np.copy(dfOrig[idx][intervals[idx][-1]:counter[idx],0:2])
            if np.linalg.norm(C-B)/np.linalg.norm(A-B)>(2-tol) and np.linalg.norm(C-B)>20:  certainty[idx] = True

# get last interval validity
for idx in range(nCameras):
    if not len(dfOrig[idx]): continue
    if certainty[idx]:
        beg,end = intervals[idx][-1],counter[idx]-1
        timeIntervals[idx].append([dfOrig[idx][beg,6],dfOrig[idx][end,6]])    
        if verbose: print('camera #'+str(idx)+' valid from '+str(round(dfOrig[idx][beg,6]/1e6,2))+'s to '+str(round(dfOrig[idx][end,6]/1e6,2))+'s')

# outcome
for i in range(nCameras): 
    print('  >> camera '+str(i)+': '+str(len(dfOrig[i]))+' valid images, missed '+str(int(missed[i]))+' images')
            
## debug received centroids  
'''for idx in range(nCameras):
    os.system('rm pics/cam'+str(idx)+'/*')
    for j in range(0,dfOrig[idx].shape[0]):
        pts,name,img,k = dfOrig[idx][j,0:6].reshape(-1,2),int(dfOrig[idx][j,6]),np.ones((540,960,3))*255,0
        for k in range(0,3):
            pt = pts.reshape(-1,2)[k]
            center = (int(np.round(pt[0]*16)),int(np.round(pt[1]*16)))
            circle(img,center,10,(255,0,0),5,shift=4)
            putText(img,str(k),(int(center[0]/16)-25, int(center[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
        imwrite('pics/cam'+str(idx+1)+'/'+str(name).zfill(20)+'.jpg',img)''';


### Computing the extrinsics parameters

Having the centroids ordered, we can interpolate all cameras to a common database and triangulate the markers within the valid time intervals. Then we estimate the fundamental and essential matrices, which can be decomposed to rotation and translation between cameras.

It is known that the best calibration result is obtained from cameras that have 90 degrees difference between them. Therefore, we only calibrated sequential pairs (i.g. (0,1), (1,2), (2,3), ...). A bundle adjustment should be added for a larger number of cameras.

In [ ]:
# initiating empty arrays
rotation,translation,scale,FMatrix,points3D_perPair,ts_perPair,firCentroids,secCentroids = [np.identity(3)],[np.zeros((1,3))],[[1]],[],[],[],[],[]
allIdx = [(i,i+1) for i in range(nCameras-1)]           # list of sequential pairs (0,1),(1,2),(2,3)...
#allIdx = list(combinations(list(range(nCameras)),2))   # list of all possible pairs

# setting up plotting function for epipolar lines
from cv2 import line
def drawlines(img1,img2,lines,pts1,pts2):
    r,c = img1.shape
    img1 = cvtColor(img1.astype('float32'),COLOR_GRAY2RGB)
    img2 = cvtColor(img2.astype('float32'),COLOR_GRAY2RGB)
    listColors = [(0,0,255),(0,255,0),(255,0,0)]
    i = 0
    for r,pt1,pt2 in zip(lines,pts1,pts2):
        color = listColors[i]
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
        img1 = line(img1, (x0,y0), (x1,y1), color,1)
        img1 = circle(img1,tuple(pt1),5,color,-1)
        img2 = circle(img2,tuple(pt2),5,color,-1)
        i+=1
    return img1,img2

# calibration loop for a pair of sequential cameras
for [m,n] in allIdx:
    # compute valid time intersection for interpolation
    intersections = [[max(first[0], second[0]), min(first[1], second[1])]  
                        for first in timeIntervals[m] for second in timeIntervals[n]  
                        if max(first[0], second[0]) <= min(first[1], second[1])]
    # create and fill inteprolation dataset based on the intersection of valid time intervals 
    dfInterp = np.zeros((int(recTime/step),2*6+1))
    dfInterp[:,-1] = np.arange(0,recTime,step)
    for [beg,end] in intersections:
        for idx in [m,n]:
            validIdx = [i for i in range(0,dfOrig[idx].shape[0]) if beg<=dfOrig[idx][i,-1]<=end]
            coord,timeNow = dfOrig[idx][validIdx,0:6],dfOrig[idx][validIdx,6]/1e6
            if timeNow.shape[0]<=2: continue
            lowBound,highBound = math.ceil(timeNow[0]/step),math.floor(timeNow[-1]/step)
            if verbose: print('interpolated #'+str(idx+1)+' from '+str(round(lowBound*step,2))+'s to '+str(round(highBound*step,2))+'s')
            tNew = np.linspace(lowBound,highBound,int((highBound-lowBound))+1,dtype=np.uint16)
            if timeNow.shape[0]<=2: continue
            ff = CubicSpline(timeNow,coord,axis=0)
            if idx == m: dfInterp[tNew,0:6] = ff(tNew*step)
            else: dfInterp[tNew,6:12] = ff(tNew*step)
    # get data from the interpolation dataset
    dfInterp = np.delete(dfInterp,np.unique([i for i in range(0,dfInterp.shape[0]) for idx in range(2) if not np.any(dfInterp[i][idx*6:idx*6+6])]),axis=0)
    if dfInterp.shape[0] < 10: 
        print('[ERROR] no valid image intersection for cameras '+str(m)+' and '+str(n))
        break
    centroids1,centroids2 = dfInterp[:,0:6].reshape(-1,2),dfInterp[:,6:12].reshape(-1,2)
    print('interpolated '+str(dfInterp.shape[0])+' images between cams '+str(m)+' and '+str(n))
    # estimate fundamental and essential matrices 
    print('[INFO] Computing fundamental and essential matrix between cameras '+str(m)+'-'+str(n))
    F,_ = estimateFundMatrix_8norm(np.array(centroids1),np.array(centroids2),verbose=verbose)
    E = np.matmul(cameraMat[n].T, np.matmul(F, cameraMat[m]))
    if verbose: print("\nEssenc. Mat.\n", E.round(4))
    # decompose to rotation and translation between cameras
    R, t = decomposeEssentialMat(E, cameraMat[m], cameraMat[n], np.array(centroids1), np.array(centroids2))
    if np.any(np.isnan(R)): print('no valid rotation matrix')
    else:
        if verbose:
            print("\nRot. Mat.\n", R.round(4))
            print("\nTrans. Mat.\n", t.round(4))
    # create projection matrices and triangulate to compute scale
    P1,P2 = np.hstack((cameraMat[m], [[0.], [0.], [0.]])),np.matmul(cameraMat[n], np.hstack((R, t.T)))
    projPt1,projPt2 = myProjectionPoints(np.array(centroids1)),myProjectionPoints(np.array(centroids2))
    points4d = triangulatePoints(P1.astype(float),P2.astype(float),projPt1.astype(float),projPt2.astype(float))
    points3d = (points4d[:3, :]/points4d[3, :]).T
    if points3d[0, 2] < 0: points3d = -points3d
    tot,L_real_AC,L_real_AB,L_real_BC,L_AC_vec,L_BC_vec,L_AB_vec,k,false_idx,time_idx = 0,15.7,5.35,10.3,[],[],[],0,[],[]
    # compute sdt deviation and plot beautiful stuff
    for [A, B, C] in points3d.reshape([-1, 3, 3]):
        L_rec_AC,L_rec_BC,L_rec_AB = np.linalg.norm(A-C),np.linalg.norm(B-C),np.linalg.norm(A-B)
        tot = tot + L_real_AC/L_rec_AC + L_real_BC/L_rec_BC + L_real_AB/L_rec_AB
        k = k + 3
        L_AC_vec.append(L_rec_AC)
        L_BC_vec.append(L_rec_BC)
        L_AB_vec.append(L_rec_AB)   
    lamb = tot/k
    print('\tScale between real world and triang. point cloud is: ', lamb.round(2))
    print('\tL_AC >> mean = ' + str((np.mean(L_AC_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_AC_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_AC_vec)*lamb-L_real_AC)))).round(4)) + "cm")
    print('\tL_AB >> mean = ' + str((np.mean(L_AB_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_AB_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_AB_vec)*lamb-L_real_AB)))).round(4)) + "cm")
    print('\tL_BC >> mean = ' + str((np.mean(L_BC_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_BC_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_BC_vec)*lamb-L_real_BC)))).round(4)) + "cm")
    # debug plot for error before refiment
    '''fig = plt.figure(figsize=(10, 6), dpi=200)
    L_AC_vec_plot,L_BC_vec_plot,L_AB_vec_plot = np.array(L_AC_vec)*lamb - L_real_AC,np.array(L_BC_vec)*lamb - L_real_BC,np.array(L_AB_vec)*lamb - L_real_AB
    plt.plot(dfInterp[:,-1],L_AC_vec_plot, 'o', label="AC")
    plt.plot(dfInterp[:,-1],L_BC_vec_plot, 'o', label="BC")
    plt.plot(dfInterp[:,-1],L_AB_vec_plot, 'o', label="AB")
    plt.axhline(y=0.0, color='r', linestyle='-',linewidth=5)
    plt.grid()
    plt.xlabel("time (seconds)",fontsize=14, fontweight='bold')
    plt.ylabel("Deviation to real value (cm)",fontsize=14,fontweight='bold')
    plt.legend()
    ax = fig.axes
    ax[0].minorticks_on()
    plt.grid(which='both')
    plt.xlim(0,recTime)
    plt.rc('xtick',labelsize=14)
    plt.rc('ytick',labelsize=14)
    plt.draw()
    plt.show()'''
    # getting indexes of images where the AC markers are further than 1% of the real distance
    points3d_new,i,k= points3d*lamb,0,0
    for [A, B, C] in points3d_new.reshape([-1, 3, 3]):
        L_reconst = np.sqrt(np.sum((A-C)**2, axis=0))
        valid = abs(L_real_AC-L_reconst)/L_real_AC < 0.01
        if not valid: 
            i = i + 1
            false_idx.extend((k,k+1,k+2))
            time_idx.append(int(k/3))
        k+=3
    print("\tImages distant more than 1% from the real value = " + str(i)+'/'+str(int(points3d.shape[0]/3)))
    # deleting points and refining estimation of the fundamental and essential matrices
    print("[INFO] Refining fundamental matrix estimation")
    centroids1,centroids2,dfInterp=np.delete(centroids1,false_idx,axis=0),np.delete(centroids2,false_idx,axis=0),np.delete(dfInterp,time_idx,axis=0)
    F,_ = estimateFundMatrix_8norm(np.array(centroids1),np.array(centroids2),verbose = 0)
    E = np.matmul(cameraMat[n].T, np.matmul(F, cameraMat[m]))
    if verbose: print("\nEssenc. Mat.\n", E.round(4))
    # decompose to rotation and translation between cameras
    R, t = decomposeEssentialMat(E, cameraMat[m], cameraMat[n], np.array(centroids1), np.array(centroids2))
    if np.any(np.isnan(R)): print('no valid rotation matrix')
    else:
        if verbose:
            print("\nRot. Mat.\n", R.round(4))
            print("\nTrans. Mat.\n", t.round(4))
    P1,P2 = np.hstack((cameraMat[m], [[0.], [0.], [0.]])),np.matmul(cameraMat[n], np.hstack((R, t.T)))
    projPt1,projPt2 = myProjectionPoints(np.array(centroids1)),myProjectionPoints(np.array(centroids2))
    points4d = triangulatePoints(P1.astype(float),P2.astype(float),projPt1.astype(float),projPt2.astype(float))
    points3d = (points4d[:3, :]/points4d[3, :]).T
    if points3d[0, 2] < 0: points3d = -points3d
    tot,L_AC_vec,L_BC_vec,L_AB_vec,k,false_idx = 0,[],[],[],0,[]
    # compute sdt deviation and plot beautiful stuff
    for [A, B, C] in points3d.reshape([-1, 3, 3]):
        L_rec_AC,L_rec_BC,L_rec_AB = np.linalg.norm(A-C),np.linalg.norm(B-C),np.linalg.norm(A-B)
        tot = tot + L_real_AC/L_rec_AC + L_real_BC/L_rec_BC + L_real_AB/L_rec_AB
        k = k + 3
        L_AC_vec.append(L_rec_AC)
        L_BC_vec.append(L_rec_BC)
        L_AB_vec.append(L_rec_AB)   
    lamb = tot/k
    print('\tScale between real world and triang. point cloud is: ', lamb.round(2))
    print('\tL_AC >> mean = ' + str((np.mean(L_AC_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_AC_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_AC_vec)*lamb-L_real_AC)))).round(4)) + "cm")
    print('\tL_AB >> mean = ' + str((np.mean(L_AB_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_AB_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_AB_vec)*lamb-L_real_AB)))).round(4)) + "cm")
    print('\tL_BC >> mean = ' + str((np.mean(L_BC_vec)*lamb).round(4)) + "cm, std. dev = " + str((np.std(L_BC_vec)*lamb).round(4)) +
        "cm, rms = " + str((np.sqrt(np.mean(np.square(np.array(L_BC_vec)*lamb-L_real_BC)))).round(4)) + "cm")
    # debug plot for error after refiment
    fig = plt.figure(figsize=(10, 6), dpi=100)
    L_AC_vec_plot,L_BC_vec_plot,L_AB_vec_plot = np.array(L_AC_vec)*lamb - L_real_AC,np.array(L_BC_vec)*lamb - L_real_BC,np.array(L_AB_vec)*lamb - L_real_AB
    plt.plot(dfInterp[:,-1],L_AC_vec_plot, 'o', label="AC")
    plt.plot(dfInterp[:,-1],L_BC_vec_plot, 'o', label="BC")
    plt.plot(dfInterp[:,-1],L_AB_vec_plot, 'o', label="AB")
    plt.axhline(y=0.0, color='r', linestyle='-',linewidth=5)
    plt.grid()
    plt.xlabel("time (seconds)",fontsize=14, fontweight='bold')
    plt.ylabel("Deviation to real value (cm)",fontsize=14,fontweight='bold')
    plt.legend()
    ax = fig.axes
    ax[0].minorticks_on()
    plt.grid(which='both')
    plt.xlim(0,recTime)
    plt.rc('xtick',labelsize=14)
    plt.rc('ytick',labelsize=14)
    plt.draw()
    plt.show()
    # saving extrinsics parameters to later use
    translation.append(t)
    rotation.append(R)
    scale.append([lamb])
    FMatrix.append(F)
    points3D_perPair.append(points3d)
    ts_perPair.append(dfInterp[:,-1])
    firCentroids.append(centroids1)
    secCentroids.append(centroids2)
    # epipolar lines plot  
    '''img1,img2,n = np.ones((720,960))*255,np.ones((720,960))*255,0
    pts1,pts2 = np.int32(centroids1[n:n+3].reshape(-1,2)),np.int32(centroids2[n:n+3].reshape(-1,2))
    lines1 = computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5,img6 = drawlines(img1,img2,lines1,pts1,pts2)
    lines2 = computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img3,img4 = drawlines(img2,img1,lines2,pts2,pts1)
    plt.figure(figsize=(20, 16),dpi=100)
    plt.subplot(121),plt.imshow(img5.astype(np.uint8))
    plt.subplot(122),plt.imshow(img3.astype(np.uint8))
    plt.show()'''
    print('--')


### Update reference to the 0th camera

With the rotation and translation in scale between cameras, we have to compound the rigid body transformations and obtain the pose of each camera in relation to the 0th camera. The triangulated 3D points must also be rotated and translated in reference to the world origin.

In [ ]:
# setting 3D plot variables
fig = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.axes(projection='3d')
ax.set_xlim(-1, 4)
ax.set_zlim(-6, 0)
ax.set_ylim(-1, 4)
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')
ax.set_xlabel('X', fontweight='bold',labelpad=15)
ax.set_ylabel('Z', fontweight='bold',labelpad=15)
ax.set_zlabel('Y', fontweight='bold',labelpad=5)
cmhot = plt.get_cmap("jet")
ax.view_init(elev=30, azim=-50) 
plt.gca().invert_zaxis()
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1., 1., .5, 1.]))
colours = [['fuchsia','plum'],['darkorange','gold'],['limegreen','greenyellow'],['blue','lightsteelblue']]

# initializing arrays
allPts3d,projMat = [],[]
dfTriang = np.zeros((int(recTime/step),(nCameras-1)*4*3+1))
dfTriang[:,-1] = np.arange(0,recTime,step)

# getting coordinates from each camera and 3D points in relation to the 0th camera
for j in range(nCameras):
    # initialize initial values of the projection matrices
    x,y,z= np.array([1, 0, 0, 0]), np.array([0, 1, 0, 0]),np.array([0, 0, 1, 0])
    P_new = np.vstack((np.hstack((np.identity(3),np.zeros((3,1)))),np.hstack((np.zeros((3)),1))))
    # iterate over the previous cameras to find the relation to the 0th camera
    # e.g. 3 -> 2 -> 1 -> 0
    for i in np.flip(range(j+1)):
        t,R,lamb = np.array(translation[i][0]).reshape(-1,3),np.array(rotation[i]),scale[i]
        t_new = np.matmul(-t, R).reshape(-1,3)*lamb/100
        P = np.vstack((np.hstack((R.T,t_new.T)),np.hstack((np.zeros((3)),1))))
        P_new = np.matmul(P,P_new)
    # save new projection matrix
    projMat.append(P_new)
    # plot camera pose
    x,y,z = np.matmul(P_new,x),np.matmul(P_new,y),np.matmul(P_new,z)
    o = np.matmul(P_new,[[0.],[0.],[0.],[1]]).ravel()
    ax.quiver(o[0], o[2], o[1], x[0], x[2], x[1], arrow_length_ratio=0.1, edgecolors="r", label='X axis')
    ax.quiver(o[0], o[2], o[1], y[0], y[2], y[1], arrow_length_ratio=0.1, edgecolors="b", label='Y axis')
    ax.quiver(o[0], o[2], o[1], z[0], z[2], z[1], arrow_length_ratio=0.1, edgecolors="g", label='Z axis')
    ax.scatter(o[0], o[2], o[1], s=50, edgecolor=colours[j][0], facecolor=colours[j][1], linewidth=2,  label = 'Camera '+str(j))
    # save triangulated 3D points in relation to the 0th camera
    if j<(nCameras-1):
        points3d = np.hstack((points3D_perPair[j]*scale[j+1][0]/100,np.ones((points3D_perPair[j].shape[0],1)))).T
        points3d = np.matmul(P_new,points3d)
        dfTriang[np.array(ts_perPair[j]*(1/step)).astype(int),j*12:(j+1)*12] = points3d.T.reshape(-1,12)
        ax.scatter(points3d[0], points3d[2], points3d[1], s=50, c=points3d[2], cmap=cmhot)
        if not len(allPts3d): allPts3d = points3d.copy()
        else: allPts3d = np.hstack((allPts3d,points3d.copy()))

# final plot commands
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),ncol=3,loc ='center',edgecolor='silver', bbox_to_anchor=(0.5, 0.8))
plt.draw()
plt.show()

### Global bundle adjustment

For a larger number of cameras, you may use optimzation to minimize the reprojection error. It would be ideal to have the same number of points between each camera pair, which could be achieved using clusterization or a weighted regression. Thus, this is a non-linear weighted sparse least squares problem. A solution was derived from [this `scipy` project](https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html).  

In [ ]:
'''from scipy.sparse import lil_matrix
from scipy.optimize import least_squares

# Computing the 2D reprojection from a 3D point >> x = PX
def reproject(points3d,cameraParams):
    camCoef,rod,t,R  = cameraParams[:,0:4],cameraParams[:,4:7].reshape(-1,3,1),cameraParams[:,7:10].reshape(-1,3,1),[]
    for r in rod: R.append(Rodrigues(r)[0])
    R = np.array(R).reshape(-1,3,3)
    nPts = camCoef.shape[0]
    K = np.transpose(np.array([[camCoef[:,0],np.zeros(nPts),camCoef[:,2]],
                  [np.zeros(nPts),camCoef[:,1],camCoef[:,3]], 
                  [np.zeros(nPts),np.zeros(nPts),np.ones(nPts)]]),(2,0,1)) 
    P = np.matmul(K,np.dstack((R, t)))
    newPts3D = np.transpose(np.hstack((points3d,np.ones((points3d.shape[0],1)))).reshape(-1,1,4),(0,2,1))
    newPts = np.matmul(P,newPts3D)
    pts2D = newPts[:,0:2,:]/newPts[:,2,:].reshape(-1,1,newPts.shape[-1])
    return np.transpose(pts2D,(0,2,1)).reshape(-1,2)

# residual computation formula >> x_reprojected - x_measured
def reprojectionError(params,nCameras,points_2d,cameraIdx,pointIdx,weights):
    cameraParams = params[:nCameras*10].reshape(-1, 10)
    points_3d = params[nCameras*10:].reshape(-1, 3)
    points_proj = reproject(points_3d[pointIdx.astype(int)], cameraParams[cameraIdx.astype(int)])
    return np.multiply(weights,points_proj - points_2d).ravel()

# sparse matrix (the 0th camera stays at the origin)
def bundle_adjustment_sparsity(nCameras,nPts,camIdx,pointIdx):
    m = camIdx.size*2
    n = nCameras*10+nPts*3
    A = lil_matrix((m, n), dtype=int)
    i = np.arange(camIdx.size)
    for s in range(4,10):
        A[2*i[int(camIdx.size/2):],10+s] = 1
        A[2*i[int(camIdx.size/2):]+1,10+s] = 1
    for s in range(3):
        A[2*i,nCameras*10+pointIdx*3+s] = 1
        A[2*i+1,nCameras*10+pointIdx*3+s] = 1
    A[:,0:10] = 0
    return A     

# initiating arrays
cameraParams,points3d,points2d,cameraIdx,pointIdx,weights = [],[],[],[],[],[]
# setting camera parameters to shape using rodrigues vectors
for i in range(nCameras): 
    K = cameraMat[i]
    P = projMat[i].copy()    
    R = P[0:3,0:3].T
    lamb = np.linalg.norm(P[0:3,3])*100
    if not lamb: t = np.matmul(-np.copy(P[0:3,3]),R.T).reshape(-1,3)
    else: t = np.matmul(-np.copy(P[0:3,3]*100/lamb),R.T).reshape(-1,3)*lamb/100
    #if i==1: P = np.matmul(K,np.hstack((R, t.T)))
    if not len(cameraParams): cameraParams=np.hstack((np.array([K[0][0],K[1][1],K[0][2],K[1][2]]).reshape(-1,4),Rodrigues(R)[0].reshape(-1,3),t.reshape(-1,3)))
    else: cameraParams=np.vstack((cameraParams,np.hstack((np.array([K[0][0],K[1][1],K[0][2],K[1][2]]).reshape(-1,4),Rodrigues(R)[0].reshape(-1,3),t.reshape(-1,3)))))
x = np.hstack((cameraParams.ravel(), allPts3d[0:3].T.ravel()))
# setting indexes and weight variables to shape
for i in range(len(allIdx)):
    new2dPts = np.vstack((firCentroids[i],secCentroids[i]))
    [m,n]=allIdx[i]
    if not len(points2d): 
        points2d = new2dPts.copy()
        cameraIdx = np.hstack((np.ones(firCentroids[i].shape[0])*m,np.ones(secCentroids[i].shape[0])*n))
        weights = np.ones(firCentroids[i].shape[0]*2)/(firCentroids[i].shape[0]*2)
        pointIdx = np.hstack(([i for i in range(firCentroids[i].shape[0])],[i for i in range(secCentroids[i].shape[0])]))
    else: 
        points2d = np.vstack((points2d,new2dPts.copy()))
        cameraIdx = np.hstack((cameraIdx,np.ones(firCentroids[i].shape[0])*m,np.ones(secCentroids[i].shape[0])*n))
        offset = pointIdx[-1]+1
        weights = np.hstack((weights,np.ones(firCentroids[i].shape[0]*2)/(firCentroids[i].shape[0]*2)))
        pointIdx = np.hstack((pointIdx,np.hstack(([i for i in range(offset,offset+firCentroids[i].shape[0])],
                                                    [i for i in range(offset,offset+secCentroids[i].shape[0])]))))
weights = (weights*cameraIdx.shape[0]).reshape(-1,1)
# Least squares (but not weighted)
r = reprojectionError(x,nCameras,points2d,cameraIdx,pointIdx,np.ones_like(weights))
A = bundle_adjustment_sparsity(nCameras,allPts3d.shape[1],cameraIdx,pointIdx)
res = least_squares(reprojectionError, x, jac_sparsity=A, verbose=0, x_scale='jac', ftol=1e-6, method='trf',
                    args=(nCameras,points2d,cameraIdx,pointIdx,np.ones_like(weights)))

# plot debug 
# plt.plot(r,label='initial residue')                    
# plt.plot(res.fun,label='optimized residue')
print('Before optimization reprojection error (in pixels): ')
print('\tMean:',r.mean())
print('\tStd. dev. :',r.std())
print('\tMin:',r.min())
print('\tMax:',r.max())
r = reprojectionError(res.x,nCameras,points2d,cameraIdx,pointIdx,np.ones_like(weights))
print('After optimization reprojection error (in pixels): ')
print('\tMean:',r.mean())
print('\tStd. dev. :',r.std())
print('\tMin:',r.min())
print('\tMax:',r.max())''';

## Calibration of ground plane

The following cell sets up the name of the CSV where the data is stored and the size of the arrays to be read. Alter to the values used with the `ground.py` code.

In [ ]:
##########################################
### YOU MAY CHANGE THE VARIABLES BELOW ###
##########################################

# name (folder/camGround.csv) where the CSV from ground.py is saved
dfCSV = np.genfromtxt('/home/debora/Desktop/MoCapRasp/server/data/camGround.csv', delimiter=',')

### Centroid ordering

As we already know the fundamental matrix between the cameras 0 and 1, we use the epipolar line approach to order the centroids. Because the wand is still, we do not need to interpolate the points. 

In [ ]:
## initiate variables
counter,i,lastTime = np.zeros(nCameras,dtype=np.int32),0,np.zeros(nCameras,dtype=np.int32)

# read from CSV
while i!=dfCSV.shape[0]:
    # get data from the line
    line = dfCSV[i]
    idx = int(line[8])
    undCoord,timeNow,imgNumber = line[0:6].reshape(-1,2),line[6],line[7]
    # save points
    if not counter[idx]: dfOrig[idx] = np.hstack((undCoord.reshape(6),timeNow))
    # update datasets variables
    i+=1
    counter[idx]+=1
    # do I have enough points? break if yes
    if np.all(counter>0): break

# order centroids
for j in range(nCameras-1):
    # collect extrinsics from calibration between camera 0 and 1
    F = FMatrix[j]
    R,t,lamb = rotation[j+1],translation[j+1].reshape(-1,3),scale[j+1]
    # order centroids per epipolar line
    pts1,pts2 = dfOrig[j][0:6].reshape(-1,2),dfOrig[j+1][0:6].reshape(-1,2)
    orderSecondFrame = getOrderPerEpiline(pts1,pts2,3,np.copy(F))
    pts2 = np.copy(pts2[orderSecondFrame])
    # save dataset
    dfOrig[j+1][0:6] = pts2.copy().ravel()
    # plot corresponding centroids 
    '''pts1,pts2 = np.int32(pts1),np.int32(pts2)
    img1,img2 = np.zeros((720,960)),np.zeros((720,960))
    lines1 = computeCorrespondEpilines(pts2.reshape(-1,1,2), 2,F)
    lines1 = lines1.reshape(-1,3)
    img5,_ = drawlines(img1,img2,lines1,pts1,pts2)
    lines2 = computeCorrespondEpilines(pts1.reshape(-1,1,2), 1,F)
    lines2 = lines2.reshape(-1,3)
    img3,_ = drawlines(img2,img1,lines2,pts2,pts1)
    plt.figure(figsize=(8, 6),dpi=100)
    plt.imshow(img5.astype(np.uint8)) 
    plt.axis('off') 
    plt.show()
    plt.figure(figsize=(8, 6),dpi=100)
    plt.imshow(img3.astype(np.uint8))
    plt.axis('off') 
    plt.show()''';

### Triangulation and plane estimation

We use the ordered centroids and triangulate using the calibrated extrinsics. We may then find the plane that passes trought the three triangulated points and the rigid body transformation to make it equal to the ground XZ plane. 

Firstly, we translate the plane `d/b` units to match the Y axis origin. After, we rotate it to meet as close as possible the plane `y = 0`. Finally we displaced all cameras so the 0th camera occupies `[0,h,0]`, for `h` the height of the camera.

In [ ]:
# triangulate ordered centroids from the first pair
pts1,pts2 = np.copy(dfOrig[0][0:6].reshape(-1,2)),np.copy(dfOrig[1][0:6].reshape(-1,2))
R,t,lamb = rotation[1],translation[1].reshape(-1,3),scale[1]
P1,P2 = np.hstack((cameraMat[0], [[0.], [0.], [0.]])),np.matmul(cameraMat[1], np.hstack((R, t.T)))
projPt1,projPt2 = myProjectionPoints(np.array(pts1)),myProjectionPoints(np.array(pts2))
points4d = triangulatePoints(P1.astype(float),P2.astype(float),projPt1.astype(float),projPt2.astype(float))
points3d = (points4d[:3, :]/points4d[3, :]).T
if points3d[0, 2] < 0: points3d = -points3d
points3d = points3d*lamb/100

# get ground plane coefficients
plane = findPlane(points3d[0],points3d[1],points3d[2])
if np.any(plane[0:3]<0): plane = findPlane(points3d[0],points3d[2],points3d[1])
[a,b,c,d]=plane
# get the orthonogal vector to the plane
v,k=np.array([a,b,c]),np.array([0,1,0])
# compute the angle between the plane and the y axis
cosPhi = np.dot(v,k)/(np.linalg.norm(v)*np.linalg.norm(k))
# compute the versors
[u1,u2,u3] = np.cross(v,k)/np.linalg.norm(np.cross(v,k))
# get the rotation matrix and new ground plane coefficients
sinPhi = np.sqrt(1-pow(cosPhi,2))
R_plane = np.array([
        [cosPhi+u1*u1*(1-cosPhi),u1*u2*(1-cosPhi)-u3*sinPhi,u2*sinPhi+u1*u3*(1-cosPhi)],
        [u1*u2*(1-cosPhi)+u3*sinPhi,cosPhi+u2*u2*(1-cosPhi),u2*u3*(1-cosPhi)-u1*sinPhi],
        [u1*u3*(1-cosPhi)-u2*sinPhi,u2*u3*(1-cosPhi)+u1*sinPhi,cosPhi+u3*u3*(1-cosPhi)]])
[A,B,C] = np.matmul(R_plane,np.array([a,b,c]).T)
newPlane = np.array([A,B,C])

# setting 3D plot variables
fig = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.axes(projection='3d')
ax.set_xlim(-1, 4)
ax.set_zlim(-6, 0)
ax.set_ylim(-1, 4)
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')
ax.set_xlabel('X', fontweight='bold',labelpad=15)
ax.set_ylabel('Z', fontweight='bold',labelpad=15)
ax.set_zlabel('Y', fontweight='bold',labelpad=5)
cmhot = plt.get_cmap("jet")
ax.view_init(elev=30, azim=-50) 
plt.gca().invert_zaxis()
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1., 1., .5, 1.]))
colours = [['fuchsia','plum'],['darkorange','gold'],['limegreen','greenyellow'],['blue','lightsteelblue']]

# initializing arrays
zDisplacement = 0
P_plane = np.vstack((np.hstack((R_plane,np.array([0,0,0]).reshape(3,-1))),np.hstack((np.zeros((3)),1))))

# plot each camera translated and rotated to meet the ground plane
for j in range(nCameras):
    o = np.matmul(projMat[j],[[0.],[0],[0.],[1]]).ravel()
    o+= [0,+d/b,0,0]
    o = np.matmul(P_plane,o).ravel()
    if not j: zDisplacement = o[2]
    o+= [0,0,-zDisplacement,0]
    x,y,z= np.array([1, 0, 0, 0]), np.array([0, 1, 0, 0]),np.array([0, 0, 1, 0])
    x,y,z = np.matmul(projMat[j],x),np.matmul(projMat[j],y),np.matmul(projMat[j],z)
    x,y,z = np.matmul(P_plane,x),np.matmul(P_plane,y),np.matmul(P_plane,z)
    ax.quiver(o[0], o[2], o[1], x[0], x[2], x[1], arrow_length_ratio=0.1, edgecolors="r", label='X axis')
    ax.quiver(o[0], o[2], o[1], y[0], y[2], y[1], arrow_length_ratio=0.1, edgecolors="b", label='Y axis')
    ax.quiver(o[0], o[2], o[1], z[0], z[2], z[1], arrow_length_ratio=0.1, edgecolors="g", label='Z axis')
    ax.scatter(o[0], o[2], o[1], s=50, edgecolor=colours[j][0], facecolor=colours[j][1], linewidth=2,  label = 'Camera '+str(j))
    
# plot the ground plane
x,z = np.linspace(-1,1,30),np.linspace(3,5,10)
X,Z = np.meshgrid(x,z)
Y=(-newPlane[0]*X -newPlane[2]*Z)/newPlane[1]
surf = ax.plot_surface(X,Z-zDisplacement,Y,color='b',alpha=.15,label="Wand's plane")
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d
# plot markers
points3d = np.hstack((points3d,np.ones((points3d.shape[0],1))))
points3d+= [0,d/b,0,0]
points3d = np.matmul(P_plane,points3d.T).T
points3d+= [0,0,-zDisplacement,0]
points3d = points3d.T
ax.scatter(points3d[0], points3d[2], points3d[1], s=50, c=points3d[2], cmap=cmhot, label= 'Markers')

# axis setup and plot variables
first3Dpoints = points3d.copy()
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),ncol=3,loc ='center',edgecolor='silver', bbox_to_anchor=(0.5, 0.8))
plt.draw()
plt.show()

### Correction matrices

Since **there are errors in the focal distance** calibration, there will be errors when matching the clouds from different pairs of camera. But we can compute the rotation and translation from each point cloud to the point cloud of the pair 0-1. 

**I opted not to use it and really see the calibration errors between the pairs.**

In [ ]:
'''# setting 3D plot variables
fig = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.axes(projection='3d')
ax.set_xlim(0.2, 0.8)
ax.set_zlim(-6, 0)
ax.set_ylim(2.8,3.2)
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')
ax.set_xlabel('X', fontweight='bold',labelpad=15)
ax.set_ylabel('Z', fontweight='bold',labelpad=15)
ax.set_zlabel('Y', fontweight='bold',labelpad=5)
cmhot = plt.get_cmap("jet")
ax.view_init(elev=30, azim=-50) 
plt.gca().invert_zaxis()
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1., 1., .5, 1.]))
colours = [['fuchsia','plum'],['darkorange','gold'],['limegreen','greenyellow'],['blue','lightsteelblue']]

# plot each camera translated and rotated to meet the ground plane
for j in range(nCameras):
    o = np.matmul(projMat[j],[[0.],[0],[0.],[1]]).ravel()
    o+= [0,+d/b,0,0]
    o = np.matmul(P_plane,o).ravel()
    o+= [0,0,-zDisplacement,0]
    x,y,z= np.array([1, 0, 0, 0]), np.array([0, 1, 0, 0]),np.array([0, 0, 1, 0])
    x,y,z = np.matmul(projMat[j],x),np.matmul(projMat[j],y),np.matmul(projMat[j],z)
    x,y,z = np.matmul(P_plane,x),np.matmul(P_plane,y),np.matmul(P_plane,z)
    ax.quiver(o[0], o[2], o[1], x[0], x[2], x[1], arrow_length_ratio=0.1, edgecolors="r", label='X axis')
    ax.quiver(o[0], o[2], o[1], y[0], y[2], y[1], arrow_length_ratio=0.1, edgecolors="b", label='Y axis')
    ax.quiver(o[0], o[2], o[1], z[0], z[2], z[1], arrow_length_ratio=0.1, edgecolors="g", label='Z axis')
    ax.scatter(o[0], o[2], o[1], s=50, edgecolor=colours[j][0], facecolor=colours[j][1], linewidth=2,  label = 'Camera '+str(j))

# plot the 3d points of the first camera
ax.scatter(first3Dpoints[0], first3Dpoints[2], first3Dpoints[1], s=50, c=first3Dpoints[2], cmap=cmhot, label= 'Markers')
correcMat = []

# plot the matching 3d points for the other cameras
for j in range(1,nCameras-1):
    # get data
    pts1,pts2 = dfOrig[j][0:6].reshape(-1,2),dfOrig[j+1][0:6].reshape(-1,2)
    F = FMatrix[j]
    R,t,lamb = rotation[j+1],translation[j+1].reshape(-1,3),scale[j+1]
    P1,P2 = np.hstack((cameraMat[j], [[0.], [0.], [0.]])),np.matmul(cameraMat[j+1], np.hstack((R, t.T)))
    projPt1,projPt2 = myProjectionPoints(np.array(pts1)),myProjectionPoints(np.array(pts2))
    # triangulate points
    points4d = triangulatePoints(P1.astype(float),P2.astype(float),projPt1.astype(float),projPt2.astype(float))
    points3d = (points4d[:3, :]/points4d[3, :]).T
    if points3d[0, 2] < 0: points3d = -points3d
    # displacement 
    points3d = np.hstack((points3d*scale[j+1][0]/100,np.ones((points3d.shape[0],1)))).T
    points3d = np.matmul(projMat[j],points3d).T
    points3d+= [0,d/b,0,0]
    points3d = np.matmul(P_plane,points3d.T).T
    points3d+= [0,0,-zDisplacement,0]
    # compute correction amtrix
    ptA,ptB = np.copy(points3d.T[0:3]),np.copy(first3Dpoints[0:3])
    R,t = findRandT(ptA,ptB)
    P = np.vstack((np.hstack((R,t.reshape(-1,1))),np.hstack((np.zeros((3)),1))))
    # save projection matrix
    correcMat.append(P)
    # rotate points and plot
    points3d = np.matmul(P,points3d.T).T
    points3d = points3d.T
    print(np.linalg.norm(points3d - first3Dpoints,axis=0)*100)
    ax.scatter(points3d[0], points3d[2], points3d[1], s=50, c=points3d[2], cmap=cmhot, label= 'Markers')

# plot the ground plane
#x,z = np.linspace(-1,1,30),np.linspace(3,5,10)
#X,Z = np.meshgrid(x,z)
#Y=(-newPlane[0]*X -newPlane[2]*Z)/newPlane[1]
#surf = ax.plot_surface(X,Z-zDisplacement,Y,color='b',alpha=.15,label="Wand's plane")
#surf._facecolors2d = surf._facecolor3d
#surf._edgecolors2d = surf._edgecolor3d

# axis setup and plot variables
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),ncol=3,loc ='center',edgecolor='silver', bbox_to_anchor=(0.5, 0.8))
plt.draw()
plt.show()''';

## Testing flight

The following cell sets up the name of the CSV where the data is stored and the size of the arrays to be read. Alter to the values used with the `capture.py` code.

In [ ]:
##########################################
### YOU MAY CHANGE THE VARIABLES BELOW ###
##########################################

# name (folder/camTest.csv) where the CSV from capture.py is saved
dfCSV = np.genfromtxt('/home/debora/Desktop/MoCapRasp/server/data/camTest.csv', delimiter=',')
recTime = 60     # recording time, in seconds
step = 1/100     # step of the interpolated DF
isDrone = False  # should I realign the point cloud to match the drones X and Y axis

# flags
saveDF = False                 # save dataframe 
whichCamera = ''               # save which camera
correctDF = False              # use correction matrices
refineCorrection = False       # refine correction matrices
saveFolder = '/home/debora/Desktop/MoCapRasp/server/data/'  # folder where to save DF
correctionFolder = '/home/debora/Desktop/MoCapRasp/server/data/'   # folder where to catch data to correction amtrices

### Correction matrices

Since **there are errors in the focal distance** calibration, there will be errors when matching the clouds from different pairs of camera. But we can compute the rotation and translation from each point cloud to the point cloud of the pair 0-1. 

**This correctio is done on a test dataset. Use the flags above to configure the algorithm**

In [ ]:
if correctDF:
    # read CSV
    dfTriang_12 = np.genfromtxt(correctionFolder+'dfTriang_12.csv', delimiter=',')
    dfTriang_23 = np.genfromtxt(correctionFolder+'dfTriang_23.csv', delimiter=',')
    dfTriang_34 = np.genfromtxt(correctionFolder+'dfTriang_34.csv', delimiter=',')

    # create all pairs dataset
    correcMat,allPermut = [],np.array(list(permutations(list(range(nCameras)))))
    dfTriang = np.zeros((int(recTime/step),(nCameras-1)*16+1))
    dfTriang[:,-1] = np.arange(0,recTime,step)
    dfTriang[np.around(dfTriang_12[:,-1]*100).astype(int),0:16] = dfTriang_12[:,:-1]
    dfTriang[np.around(dfTriang_23[:,-1]*100).astype(int),16:32] = dfTriang_23[:,:-1]
    dfTriang[np.around(dfTriang_34[:,-1]*100).astype(int),32:48] = dfTriang_34[:,:-1]

    # minimize pair 1 and 2
    emptyLines = np.unique([i for i in range(0,dfTriang.shape[0]) for k in [0,1] if not dfTriang[i][k*16]])
    #emptyLines = np.unique(np.hstack((emptyLines,list(range(0,2000)))))
    dfMinimize = np.delete(dfTriang,emptyLines,axis=0)
    for i in range(dfMinimize.shape[0]):
        ptsA,ptsB =  dfMinimize[i,0:16].reshape(-1,4).T,dfMinimize[i,16:32].reshape(-1,4).T
        allDists = []
        for order in allPermut:
            aux = ptsB[:,order]
            allDists.append(np.sum(np.linalg.norm(ptsA-aux,axis=0))  )
        order = allPermut[np.argmin(allDists)]
        #print(order,min(allDists))
        dfMinimize[i,16:32] = ptsB[:,order].T.reshape(-1,16)
    ptsA = dfMinimize[:,0:16].reshape(-1,4).T
    ptsB = dfMinimize[:,16:32].reshape(-1,4).T
    ptA,ptB = np.copy(ptsB[0:3]), np.copy(ptsA[0:3])    
    R,t = findRandT(ptA,ptB)
    P = np.vstack((np.hstack((R,t.reshape(-1,1))),np.hstack((np.zeros((3)),1))))
    correcMat.append(P)
    points3d = np.matmul(P,ptsB)
    error,deleteIdx = np.linalg.norm(points3d - ptsA,axis=0)*100,[]
    if refineCorrection:
        for i in range(1,len(error)):
            if abs(error[i]-np.mean(error)) > 3*np.std(error) : deleteIdx.append(int(i/4))
        if len(deleteIdx):
            dfMinimize = np.delete(dfMinimize,np.unique(deleteIdx),axis=0)
            ptsA = dfMinimize[:,0:16].reshape(-1,4).T
            ptsB = dfMinimize[:,16:32].reshape(-1,4).T
            ptA,ptB = np.copy(ptsB[0:3]), np.copy(ptsA[0:3])
            R,t = findRandT(ptA,ptB)
            P = np.vstack((np.hstack((R,t.reshape(-1,1))),np.hstack((np.zeros((3)),1))))
            correcMat.append(P)
            points3d = np.matmul(P,ptsB)
            error = np.linalg.norm(points3d - ptsA,axis=0)*100
    print('## Pair 1-2 and 2-3 ##')
    print('Using '+str(int(ptsA.shape[1]/4))+' images')
    print('min. error:', round(min(error),2),' cm')
    print('max. error:', round(max(error),2),' cm')
    print('mean error:', round(np.mean(error),2),' cm')
    print('std. dev. error:', round(np.std(error),2),' cm')
    plt.plot(error)    
    #np.savetxt(correctionFolder+'dfMinimize_12.csv',dfMinimize,delimiter=',')

    # minimize pair 1 and 3
    emptyLines = np.unique([i for i in range(0,dfTriang.shape[0]) for k in [0,2] if not dfTriang[i][k*16]])
    #emptyLines = np.unique(np.hstack((emptyLines,list(range(0,2000)))))
    dfMinimize = np.delete(dfTriang,emptyLines,axis=0)
    for i in range(dfMinimize.shape[0]):
        ptsA,ptsB =  dfMinimize[i,0:16].reshape(-1,4).T,dfMinimize[i,32:48].reshape(-1,4).T
        allDists = []
        for order in allPermut:
            aux = ptsB[:,order]
            allDists.append(np.sum(np.linalg.norm(ptsA-aux,axis=0))  )
        order = allPermut[np.argmin(allDists)]
        #print(order,min(allDists))
        dfMinimize[i,32:48] = ptsB[:,order].T.reshape(-1,16)
    ptsA = dfMinimize[:,0:16].reshape(-1,4).T
    ptsB = dfMinimize[:,32:48].reshape(-1,4).T
    ptA,ptB = np.copy(ptsB[0:3]), np.copy(ptsA[0:3])
    R,t = findRandT(ptA,ptB)
    P = np.vstack((np.hstack((R,t.reshape(-1,1))),np.hstack((np.zeros((3)),1))))
    correcMat.append(P)
    points3d = np.matmul(P,ptsB)
    error,deleteIdx = np.linalg.norm(points3d - ptsA,axis=0)*100,[]  
    if refineCorrection:
        for i in range(1,len(error)):
            if abs(error[i]-np.mean(error)) > 3*np.std(error) : deleteIdx.append(int(i/4))
        if len(deleteIdx):
            dfMinimize = np.delete(dfMinimize,np.unique(deleteIdx),axis=0)
            ptsA = dfMinimize[:,0:16].reshape(-1,4).T
            ptsB = dfMinimize[:,32:48].reshape(-1,4).T
            ptA,ptB = np.copy(ptsB[0:3]), np.copy(ptsA[0:3])
            R,t = findRandT(ptA,ptB)
            P = np.vstack((np.hstack((R,t.reshape(-1,1))),np.hstack((np.zeros((3)),1))))
            correcMat.append(P)
            points3d = np.matmul(P,ptsB)
            error = np.linalg.norm(points3d - ptsA,axis=0)*100
    print('\n## Pair 1-2 and 3-4 ##')
    print('Using '+str(int(ptsA.shape[1]/4))+' images')
    print('min. error:', round(min(error),2),' cm')
    print('max. error:', round(max(error),2),' cm')
    print('mean error:', round(np.mean(error),2),' cm')
    print('std. dev. error:', round(np.std(error),2),' cm')
    plt.plot(error) 
    #np.savetxt(correctionFolder+'dfMinimize_13.csv',dfMinimize,delimiter=',')
    

### Reading coordinates, ordering and triangulating markers

Now we do ordering and triangulation inside the reading loop. The epiline approach for ordering tests 16 possibilities at each loop, so we keep track of the camera pairs that needs ordering of the centroids. When at least one image is ordered, the blobs in following ones are tracked by proximity. The reference is reset if more than 10 images are invalid (per time missmatch or occlusion)

Nevertheless, we can only relate the two cameras when all four markers are visible at the same time. We do as following:

- Wait for the cameras to have at least 3 images in storage for the actual valid interval.
- Check which cameras still need to be related to the one that sent the last package.
- If yes, check if valid time interval of this other camera has intesection with that camera that sent the last package.
- If yes, we order the blobs at the first intersection timestamp per epipolar line.
- Then we flip the blobs according to this order on the second camera (sorted by index) in the non interpolated database since the beggining fo the actual valid time interval.
- We notify that any camera index comming after this second camera index also needs reordering when the next package is received.

Then, for triangulation, we follow as:

- Wait for the cameras to have at least 10 images in storage for the actual valid interval to start triangulating
- Interpolate the blobs that have not been already interpolated.
- Check if any of the interpolated images are also seen by any other camera (always neighbours).
- Triangulate markers with this camera (including projection matrix, ground plane rotation and correction matrix).


In [ ]:
nMarkers = 4

# initiating empty arrays
counter,i,lastTime = np.zeros(nCameras,dtype=np.int32),0,np.zeros(nCameras,dtype=np.int32)
missed,invalid,swap = np.zeros(nCameras,dtype=np.int32),np.zeros(nCameras,dtype=np.int32),np.zeros(nCameras,dtype=np.int32)
lastImgNumber = np.zeros(nCameras,dtype=np.int32)
intervals,lastTnew = [],[]

# warmUp = number of images in storage for the actual valid interval to start triangulating
# nPrevious = number of images in camera storage for the actual valid interval
dfOrig,nPrevious,warmUp = [],3,10 # warm up > n previous 

needsOrder = createNeedsOrder(nCameras,relateLast2First=0)

allPoints3d = [] # List of all 3D points in the whole footage

verbose = False # Verbose Info

for _ in range(nCameras): # For each camera
    dfOrig.append([]) # List for each camera
    intervals.append([]) # List for each camera
    lastTnew.append([]) # List for each camera

# Each PTS will have a 2D marker coordinate for each camera + 2 additional columns
dfInterp = np.zeros((int(recTime/step),nCameras*(2*nMarkers)+2)) # Last two columns are special

dfInterp[:,-2] = np.arange(0,recTime,step) # Penultimate column is filled with the time stamps
dfInterp[:,-1] = np.zeros_like(dfInterp[:,-1],dtype=np.bool8) # Last column is filled with bool zeros (false)

# Each PTS will have...
dfTriang = np.zeros((int(recTime/step),(4*nMarkers)+1)) # 4D coordinates of each marker + PTS
dfTriang[:,-1] = np.arange(0,recTime,step) # Last column is filled with the PTSs

timeArray = []
otherCamera = findOtherCamera(whichCamera,nCameras)

# Test loop done in real time
while i != dfCSV.shape[0]: # Capture frames
    start = time.time() # Just to measure loop time
    line = dfCSV[i] # Current frame info
    # Get data from the line
    # Lines are: 2D Coordinates from each marker | Current time | Image number | Camera Index
    undCoord,timeNow,imgNumber, idx = line[0:(2*nMarkers)].reshape(-1,2), line[-3], line[-2], int(line[-1])

    i += 1 # Update to next frame for next iteration

    if len(otherCamera): # If there is camera available in otherCamera
        if idx in otherCamera: continue # If Camera is in otherCamera

    # Time missmatch
    if counter[idx]:
        if abs(timeNow-lastTime[idx])>1e9: 
            if verbose: print('time missmatch')
            missed[idx] += 1 # One more missed image for the camera
            invalid[idx] += 1 # One more invalid image for the camera
            continue
    
    # Check if sequence is valid
    if imgNumber > lastImgNumber[idx] + 1: invalid[idx] = imgNumber-lastImgNumber[idx]

    # Check occlusion and order markers per proximity
    if not occlusion(undCoord,5) and not np.any(undCoord<0): 
        if invalid[idx]>=10 or not counter[idx]: 
            if verbose: print('reseting at camera', idx,', counter',counter[idx],',',timeNow/1e6,'s')
            prev,needsOrder = [],activateNeedsOrder(nCameras,idx,needsOrder,relateLast2First=0)
            intervals[idx].append(counter[idx])
        else:
            if not (counter[idx]-1): prev = np.array(dfOrig[idx][0:(2*nMarkers)]).reshape(-1,2)
            else: prev = np.array(dfOrig[idx][-1,0:(2*nMarkers)]).reshape(-1,2)
            newOrder = getTheClosest(undCoord.reshape(-1,2),prev.reshape(-1,2))
            undCoord = np.copy(undCoord[newOrder])
    else: 
        if verbose: print('not collinear or equal centroids')
        missed[idx] += 1 # One more missed image for the camera
        invalid[idx] += 1 # One more invalid image for the camera
        continue

    # Update loop variables
    lastTime[idx],lastImgNumber[idx],invalid[idx] = timeNow,imgNumber,0    
    if not counter[idx]: dfOrig[idx] = np.hstack((undCoord.reshape(2*nMarkers),timeNow))
    else: dfOrig[idx] = np.vstack((dfOrig[idx],np.hstack((undCoord.reshape(2*nMarkers),timeNow))))
    counter[idx] += 1

    # Interpolate
    if needsOrder[str(idx)].size:
        # Get if there are enough points at the valid interval
        for otherIdx in needsOrder[str(idx)]:
            if not len(intervals[otherIdx]): continue
            myCounter,myIntervals = np.array([counter[idx],counter[otherIdx]]),np.array([intervals[idx][-1],intervals[otherIdx][-1]])            
            if np.all(myCounter-myIntervals>=nPrevious):
                # see if there are intersection between arrays
                ts1,ts2 = dfOrig[idx][intervals[idx][-1]:counter[idx],(2*nMarkers)]/1e6,dfOrig[otherIdx][intervals[otherIdx][-1]:counter[otherIdx],(2*nMarkers)]/1e6
                validIdx1 = [k for k in range(0,len(ts1)) if max(ts1[0], ts2[0])-0.01<=ts1[k]<=min(ts1[-1], ts2[-1])+0.01]
                validIdx2 = [k for k in range(0,len(ts2)) if max(ts1[0], ts2[0])-0.01<=ts2[k]<=min(ts1[-1], ts2[-1])+0.01]
                # if there is intersection, get order
                if len(validIdx1) and len(validIdx2):
                    ts1,ts2 = np.copy(ts1[validIdx1]),np.copy(ts2[validIdx2])
                    if ts1.shape[0]<2 or ts2.shape[0]<2: continue
                    coord1,coord2 = dfOrig[idx][intervals[idx][-1]:counter[idx],0:(2*nMarkers)],dfOrig[otherIdx][intervals[otherIdx][-1]:counter[otherIdx],0:(2*nMarkers)]
                    coord1,coord2 = np.copy(coord1[validIdx1]),np.copy(coord2[validIdx2])
                    # Get interpolated data
                    interp1,tNew1 = myInterpolate(coord1,ts1,step)
                    interp2,tNew2 = myInterpolate(coord2,ts2,step)
                    if not len(interp1) or not len(interp2): continue
                    # Get min and max idx
                    minIdx,maxIdx = min(idx,otherIdx),max(idx,otherIdx)
                    # Get common idx
                    F = FMatrix[minIdx]
                    interpolateIdx1,interpolateIdx2 = np.argmax(np.in1d(tNew1, tNew2)),np.argmax(np.in1d(tNew2, tNew1))
                    # Order per epipolar line
                    if idx < otherIdx: orderSecondFrame,ret = getOrderPerEpiline(interp1[-1],interp2[-1],nMarkers,F,0,1)
                    else: orderSecondFrame,ret = getOrderPerEpiline(interp2[-1],interp1[-1],nMarkers,F,0,1)
                    if not ret: 
                        if verbose: print(minIdx,maxIdx,'could not rearange at',tNew2[-1]*step,'s')
                        continue
                    # Get interval to rearrange
                    if verbose: print(minIdx,maxIdx,'rearanging interval',[intervals[maxIdx][-1],counter[maxIdx]], 'to', orderSecondFrame, 'at',tNew2[-1]*step,'s')
                    # Flip blobs
                    for k in range(intervals[maxIdx][-1],counter[maxIdx]):
                       dfOrig[maxIdx][k,0:(2*nMarkers)] = np.copy(dfOrig[maxIdx][k,0:(2*nMarkers)].reshape(-1,2)[orderSecondFrame].reshape(-(2*nMarkers)))
                    # Change ordering boolean
                    needsOrder=popNeedsOrder(idx,otherIdx,needsOrder)
                    for k in range(maxIdx+1,nCameras):
                        needsOrder=activateNeedsOrder(nCameras,k,needsOrder,relateLast2First=0) 

    if (counter[idx]-intervals[idx][-1]) >= warmUp: 
        # Get data to interpolate
        coord,ts = dfOrig[idx][(counter[idx]-warmUp):counter[idx],0:(2*nMarkers)],dfOrig[idx][(counter[idx]-warmUp):counter[idx],(2*nMarkers)]/1e6
        if not len(ts): continue
        lowBound,highBound = math.ceil(ts[0]/step),math.floor(ts[-1]/step)
        # Avoid repeating intepolation
        if lastTnew[idx]:
            if lowBound<lastTnew[idx]: lowBound = lastTnew[idx]+1
        # Interpolate
        tNew = np.linspace(lowBound,highBound,int((highBound-lowBound))+1,dtype=np.uint16)
        ff = CubicSpline(ts,coord,axis=0)
        # Save data
        dfInterp[tNew,int(idx*(2*nMarkers)):int(idx*(2*nMarkers)+(2*nMarkers))] = ff(tNew*step)
        lastTnew[idx] = tNew[-1]
        # Compare if there is another camera available
        for k in tNew:
            if dfInterp[k,-1]: continue
            otherIdx = getOtherValidIdx(dfInterp[k,:],nMarkers,idx)      
            if not len(otherIdx) or otherIdx in needsOrder[str(idx)]: continue      
            # Get index from the other camera
            dfInterp[k,-1],otherIdx = True,otherIdx[0]
            # Comparing the indexes to set the projection matrices
            minIdx,maxIdx = min(idx,otherIdx),max(idx,otherIdx)
            # Getting the data
            pts1,pts2 = dfInterp[k,minIdx*(2*nMarkers):(minIdx+1)*(2*nMarkers)].reshape(-1,2),dfInterp[k,maxIdx*(2*nMarkers):(maxIdx+1)*(2*nMarkers)].reshape(-1,2)
            R,t,lamb = rotation[maxIdx],translation[maxIdx].reshape(-1,3),scale[maxIdx]
            P1,P2 = np.hstack((cameraMat[minIdx], [[0.], [0.], [0.]])),np.matmul(cameraMat[maxIdx], np.hstack((R, t.T)))
            projPt1,projPt2 = myProjectionPoints(np.array(pts1)),myProjectionPoints(np.array(pts2))
            # Triangulate
            points4d = triangulatePoints(P1.astype(float),P2.astype(float),projPt1.astype(float),projPt2.astype(float))
            points3d = (points4d[:3, :]/points4d[3, :]).T
            if points3d[0, 2] < 0: points3d = -points3d  
            # Project in scale regarding the minimum index camera
            points3d = np.hstack((points3d*lamb/100,np.ones((points3d.shape[0],1)))).T
            points3d = np.matmul(projMat[minIdx],points3d).T
            # Rotate to ground plane
            points3d += [0,d/b,0,0]
            points3d = np.matmul(P_plane,points3d.T).T
            points3d += [0,0,-zDisplacement,0]
            
            # Save to array
            dfTriang[k,0:(4*nMarkers)] = np.copy(points3d.ravel()) 
            if not len(allPoints3d): allPoints3d = np.copy(points3d)
            else: allPoints3d = np.vstack((allPoints3d,np.copy(points3d))) 

    timeArray.append(time.time()-start)

# just comprising dataset if wanted to plot
emptyLines = np.unique([i for i in range(0,dfInterp.shape[0]) if not dfInterp[i][-1]])
dfInterp = np.delete(dfInterp,emptyLines,axis=0)
dfTriang = np.delete(dfTriang,emptyLines,axis=0)
print('[INFO] found ' +str(dfTriang.shape[0])+ ' interpolated pics')
# time plot for further temporal analysis
print('[INFO] Temporal analysis')
print('\t> Min:',round(np.min(timeArray),5),'s')
print('\t> Mean:',round(np.mean(timeArray),5),'s')
print('\t> Max:',round(np.max(timeArray),5),'s')
print('\t> Std. Dev.:',round(np.std(timeArray),5),'s')
allPoints3d = np.array(allPoints3d).T
# save dataset
if saveDF:
    name = 'dfTriang_'
    if len(whichCamera): name += whichCamera
    else: name += 'Mix'
    if correctDF: name += '_corrected'
    np.savetxt(saveFolder+name+'.csv',dfTriang,delimiter=',')

# plot debug for interpolated dataset
'''for idx in range(nCameras):
    os.system('del -rf pics\cam'+str(idx+1)+'\*jpg')
    for j in range(0,dfInterp.shape[0]):
        pts,name,img,k = dfInterp[j,int(idx*8):int(idx*8+8)].reshape(-1,2),int(dfInterp[j,-2]/step),np.ones((720,960,3))*255,0
        for k in range(0,4):
            pt = pts.reshape(-1,2)[k]
            center = (int(np.round(pt[0]*16)),int(np.round(pt[1]*16)))
            circle(img,center,10,(255,0,0),5,shift=4)
            putText(img,str(k),(int(center[0]/16)-25, int(center[1]/16)-25),FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2) 
        imwrite('pics/cam'+str(idx+1)+'/'+str(name).zfill(20)+'.jpg',img)''';

### Visual plot and anaylisis

The following cells plot the 3D mat and 2D analysis of the data captured during the test live loop.

In [ ]:
# setting 3D plot variables
fig = plt.figure(figsize=(8, 8),dpi=100)
ax = plt.axes(projection='3d')
ax.set_xlim(-1, 4)
ax.set_zlim(-6, 0)
ax.set_ylim(-1, 4)
ax.set_xlabel('X')
ax.set_ylabel('Z')
ax.set_zlabel('Y')
ax.set_xlabel('X', fontweight='bold',labelpad=15)
ax.set_ylabel('Z', fontweight='bold',labelpad=15)
ax.set_zlabel('Y', fontweight='bold',labelpad=5)
cmhot = plt.get_cmap("jet")
ax.view_init(elev=30, azim=-50) 
plt.gca().invert_zaxis()
ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1., 1., .5, 1.]))
colours = [['fuchsia','plum'],['darkorange','gold'],['limegreen','greenyellow'],['blue','lightsteelblue']]

# plot each camera translated and rotated to meet the ground plane
for j in range(nCameras):
    o = np.matmul(projMat[j],[[0.],[0],[0.],[1]]).ravel()
    o+= [0,+d/b,0,0]
    o = np.matmul(P_plane,o).ravel()
    o+= [0,0,-zDisplacement,0]
    x,y,z= np.array([1, 0, 0, 0]), np.array([0, 1, 0, 0]),np.array([0, 0, 1, 0])
    x,y,z = np.matmul(projMat[j],x),np.matmul(projMat[j],y),np.matmul(projMat[j],z)
    x,y,z = np.matmul(P_plane,x),np.matmul(P_plane,y),np.matmul(P_plane,z)
    ax.quiver(o[0], o[2], o[1], x[0], x[2], x[1], arrow_length_ratio=0.1, edgecolors="r", label='X axis')
    ax.quiver(o[0], o[2], o[1], y[0], y[2], y[1], arrow_length_ratio=0.1, edgecolors="b", label='Y axis')
    ax.quiver(o[0], o[2], o[1], z[0], z[2], z[1], arrow_length_ratio=0.1, edgecolors="g", label='Z axis')
    ax.scatter(o[0], o[2], o[1], s=50, edgecolor=colours[j][0], facecolor=colours[j][1], linewidth=2,  label = 'Camera '+str(j))

# plot points after correction matrix
ax.scatter(allPoints3d[0], allPoints3d[2], allPoints3d[1], s=50, c=allPoints3d[2], cmap=cmhot, label= 'Markers')

# axis setup and plot variables
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(),ncol=3,loc ='center',edgecolor='silver', bbox_to_anchor=(0.5, 0.8))
plt.draw()
plt.show()

# save array of each point for later use (e.g. Matlab)
for k in range(0,4):
    idx = np.array(range(k,allPoints3d.shape[1],4)).astype(int)
    points3dNew = allPoints3d.T[idx]
    #np.savetxt('data/points3d'+str(k)+'.csv',points3dNew,delimiter=',')

### Rotating the point cloud and ploting

If we captured data of a drone, we can find the two markers on the front propellers and realign the X and Y axis to match the inertial body frame of the first drone position. 

In [ ]:
if isDrone:
    #variable initialization
    allCombinationsOf2 = np.array(list(combinations(list(range(0,4)),2)))
    myNorm = []
    # get first position
    pts = dfTriang[0,0:-1].reshape(-1,4)[:,0:3]
    print(pts)
    for idx in allCombinationsOf2:
        myNorm.append(np.linalg.norm(np.diff(pts[idx],axis=0)))
    # The farthest apert are the vertices A and C
    orderedNorm = np.argsort(myNorm)
    extremitiesIdx = allCombinationsOf2[orderedNorm[-1]]
    # add the non aligned marker
    allVertices = np.unique([allCombinationsOf2[orderedNorm[-1]],allCombinationsOf2[orderedNorm[-2]]])
    # find the non aligned marker index
    rightAngleIdx = allVertices[np.argmin(np.in1d(allVertices,extremitiesIdx))]
    # find the middle marker and atribute it as origin
    middleIdx = np.array([0,1,2,3])[np.argmin(np.in1d(np.array([0,1,2,3]),allVertices))]
    newO = pts[middleIdx].copy()
    # rotate the points so the the vector between the middle marker and the non aligned marker is the X axis
    transPts = pts-pts[middleIdx]
    v = (transPts[rightAngleIdx])
    k = np.array([1,0,0])*.15
    cosPhi = np.dot(v,k)/(np.linalg.norm(v)*np.linalg.norm(k))
    [u1,u2,u3] = np.cross(v,k)/np.linalg.norm(np.cross(v,k))
    sinPhi = np.sqrt(1-pow(cosPhi,2))
    R1 = np.array([
            [cosPhi+u1*u1*(1-cosPhi),u1*u2*(1-cosPhi)-u3*sinPhi,u2*sinPhi+u1*u3*(1-cosPhi)],
            [u1*u2*(1-cosPhi)+u3*sinPhi,cosPhi+u2*u2*(1-cosPhi),u2*u3*(1-cosPhi)-u1*sinPhi],
            [u1*u3*(1-cosPhi)-u2*sinPhi,u2*u3*(1-cosPhi)+u1*sinPhi,cosPhi+u3*u3*(1-cosPhi)]])
    rotPts = np.matmul(R1,transPts.T).T
    # plot
    fig = plt.figure(figsize=(8, 8),dpi=100)
    ax = plt.axes(projection='3d')
    ax.set_xlim(0.2, 0.8)
    ax.set_zlim(-6, 0)
    ax.set_ylim(2.8,3.2)
    ax.set_xlabel('X')
    ax.set_ylabel('Z')
    ax.set_zlabel('Y')
    ax.set_xlabel('X', fontweight='bold',labelpad=15)
    ax.set_ylabel('Z', fontweight='bold',labelpad=15)
    ax.set_zlabel('Y', fontweight='bold',labelpad=5)
    cmhot = plt.get_cmap("jet")
    ax.view_init(elev=30, azim=-50) 
    plt.gca().invert_zaxis()
    ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1., 1., .5, 1.]))
    colours = [['fuchsia','plum'],['darkorange','gold'],['limegreen','greenyellow'],['blue','lightsteelblue']]
    
    # find the marker to the right of the X axis
    '''vertixIdx = np.argsort(rotPts[:,2])[0]
    # find the middle point between the the marker to the right of the X axis and
    # the non aligned marker is the X axis
    middlePt = (transPts[rightAngleIdx]-transPts[vertixIdx])/2
    # rotating around Y axis to align to the middle point projection at Y=0
    k = np.array([1,0,0])*.15
    v = (middlePt-transPts[middleIdx]).copy()
    v[1] = 0
    cosPhi = np.dot(v,k)/(np.linalg.norm(v)*np.linalg.norm(k))
    [u1,u2,u3] = np.cross(v,k)/np.linalg.norm(np.cross(v,k))
    sinPhi = np.sqrt(1-pow(cosPhi,2))
    R1 = np.array([
            [cosPhi+u1*u1*(1-cosPhi),u1*u2*(1-cosPhi)-u3*sinPhi,u2*sinPhi+u1*u3*(1-cosPhi)],
            [u1*u2*(1-cosPhi)+u3*sinPhi,cosPhi+u2*u2*(1-cosPhi),u2*u3*(1-cosPhi)-u1*sinPhi],
            [u1*u3*(1-cosPhi)-u2*sinPhi,u2*u3*(1-cosPhi)+u1*sinPhi,cosPhi+u3*u3*(1-cosPhi)]])
    rotPts = np.matmul(R1,transPts.T).T
    v = np.matmul(R1,v.T).T
    middlePt = np.matmul(R1,middlePt.T).T

    # save new dataset
    allPts = []
    for n in range(0,dfTriang.shape[0]):
        pts,newPts = dfTriang[n,0:-1].reshape(-1,4)[:,0:3],[]
        transPts = pts - newO
        rotPts = np.matmul(R1,transPts.T).T
        if not len(allPts): allPts = np.array(rotPts)
        else: allPts = np.vstack((allPts,np.array(rotPts)))
        dfTriang[n,0:-1] = np.hstack((rotPts,np.ones((4,1)))).ravel()
    allPts = allPts.T

    # save array of each point for later use (e.g. Matlab)
    for k in range(0,4):
        idx = np.array(range(k,allPts.shape[1],4)).astype(int)
        points3dNew = allPts.T[idx]'''
        #np.savetxt('data/points3d'+str(k)+'_drone.csv',points3dNew,delimiter=',')

In [ ]:
'''# initialize variables
allCombinationsOf2 = np.array(list(combinations(list(range(0,4)),2)))
points3d_rightTriangle = []
points3d_centreOfMass = []
deleteIdx = []

# get position per axis
for k in range(0,dfTriang.shape[0]):
    # initialize variables
    pts,myNorm,ts = dfTriang[k,0:-1].reshape(-1,4)[:,0:3],[],dfTriang[k,-1]
    # get the distance between blobs
    for idx in allCombinationsOf2:
        myNorm.append(np.linalg.norm(np.diff(pts[idx],axis=0)))
    orderedNorm = np.argsort(myNorm)
    # the blobs furthest apart are the vertices of the triangle
    myIdx = np.unique([allCombinationsOf2[orderedNorm[-1]],allCombinationsOf2[orderedNorm[-2]]])
    centreOfMass = [i for i in range(0,4) if i not in myIdx]
    # save do array
    if not len(points3d_rightTriangle): 
         points3d_centreOfMass = pts[centreOfMass]
         points3d_rightTriangle = pts[myIdx]
    else: 
        points3d_centreOfMass = np.vstack((points3d_centreOfMass,pts[centreOfMass]))
        points3d_rightTriangle = np.vstack((points3d_rightTriangle,pts[myIdx]))

# plotting variables
fig, axs = plt.subplots(1,3,figsize=(24,4),dpi=100)
colour = ['r','b','g','y']
label = ['Position in X-axis (cm)','Position in Y-axis (cm)','Position in Z-axis (cm)']
for k in range(0,3):
    axs[k].plot(dfTriang[:,-1],points3d_centreOfMass[:,k]*100,
                'o',c=colour[k],linewidth=1)
    axs[k].set_xlim((0,recTime))
    axs[k].grid(True)
    axs[k].set_ylabel(label[k],fontweight='bold',size=14)
    axs[k].set_xlabel('Time (s)',fontweight='bold',size=14)
    plt.rcParams['xtick.labelsize']=14
    plt.rcParams['ytick.labelsize']=14

# get orientation
myOrientation = []
for k in range(0,int(points3d_rightTriangle.shape[0]/3)):
    # get the coordinates of the vertices of the triangles
    pts = points3d_rightTriangle[k*3:k*3+3]-points3d_centreOfMass[k]
    # get orthogonal vector and angles
    n = np.cross(pts[1]-pts[0], pts[2]-pts[0])
    nNormalised = n / np.linalg.norm(n)
    angles = np.rad2deg(np.arcsin(nNormalised))
    # save array
    if not len(myOrientation): myOrientation = [angles]
    else: myOrientation = np.vstack((myOrientation,[angles]))

# plotting variables
fig, axs = plt.subplots(1,3,figsize=(24,4),dpi=100)
colour = ['r','b','g']
label = [r'$\mathbf{\theta\;(deg)}$',r'$\mathbf{\phi\;(deg)}$',r'$\mathbf{\psi\;(deg)}$']
for k in range(0,3):
    axs[k].plot(dfTriang[:,-1],myOrientation[:,k],'o',c=colour[k],linewidth=3)
    axs[k].set_xlim((0,recTime))
    axs[k].grid(True)
    axs[k].set_ylabel(label[k],size=14,labelpad=3)
    axs[k].set_xlabel('Time (s)',fontweight='bold',size=14)
    plt.rcParams['xtick.labelsize']=14
    plt.rcParams['ytick.labelsize']=14

# ploting upper and lateral map
fig, axs = plt.subplots(1,2,figsize=(16,4),dpi=100)
axs[0].plot(points3d_centreOfMass[:,0]*100,points3d_centreOfMass[:,2]*100, 'o',c='orange',linewidth=3,label='trajectory')
axs[0].plot(points3d_centreOfMass[1,0]*100,points3d_centreOfMass[1,2]*100, '*',
            c='blueviolet',linewidth=3,label='take off', markersize=16)
axs[0].plot(points3d_centreOfMass[-1,0]*100,points3d_centreOfMass[-1,2]*100, 'X',
            c='green',linewidth=3,label='land', markersize=14)
axs[0].grid(True); axs[0].axis('equal')
#axs[0].set_xlim((-40,40)); axs[0].set_ylim((-50,10)); 
axs[0].set_ylabel('Position in Z-axis (cm)',fontweight='bold',size=12)
axs[0].set_xlabel('Position in X-axis (cm)',fontweight='bold',size=12)
pos = axs[0].get_position()
axs[0].set_position([pos.x0, pos.y0, pos.width * 0.8, pos.height])
axs[1].plot(points3d_centreOfMass[:,0]*100,-points3d_centreOfMass[:,1]*100, 'o',c='orange',label='Trajectory')
axs[1].plot(points3d_centreOfMass[1,0]*100,-points3d_centreOfMass[1,1]*100, '*',
            c='blueviolet',linewidth=3,label='Take off', markersize=16)
axs[1].plot(points3d_centreOfMass[-1,0]*100,-points3d_centreOfMass[-1,1]*100, 'X',
            c='green',linewidth=3,label='Land', markersize=14)
axs[1].grid(True);
#axs[1].set_xlim((-40,40)); axs[1].set_ylim((-10,110))
axs[1].set_ylabel('Position in Y-axis (cm)',fontweight='bold',size=12)
axs[1].set_xlabel('Position in X-axis (cm)',fontweight='bold',size=12)
pos = axs[1].get_position()
axs[1].set_position([pos.x0-pos.x0*0.15, pos.y0, pos.width * 0.8, pos.height])
plt.rcParams['xtick.labelsize']=14; plt.rcParams['ytick.labelsize']=14
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels,loc='center right', bbox_to_anchor=(.74, 0.28),fontsize=12,labelspacing=.8)'''